# Triplet Loss Pytorch

In this notebook I demonstrate how to train a Siamese Net with Triplet Loss. I use automatic mixed presicion to speed up training.  

Triplet loss requires anchors, positives and negatives examples. This notebook demonstrates how to prepare the dataset: https://www.kaggle.com/njelicic/tripletloss-pytorch-data-preparation

Inference: WIP

**Room for improvement:**
* Larger sample size
* Larger model 
* Longer training


In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [ ]:
SAMPLE_SIZE = 1000 # Due to long GPU runtime, this notebook only demonstrates training on a handfull of examples

In [ ]:
import timm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm.notebook import tqdm
import numpy as np 
import pandas as pd 
import sqlite3
import warnings
import cv2
warnings.filterwarnings("ignore")
import concurrent
import os

torch.cuda.empty_cache()

torch.backends.cudnn.benchmark = True

In [ ]:
anchor_pos_neg = pd.read_csv('../input/tripletloss-pytorch-data-preparation/anchor_pos_neg.csv').sample(SAMPLE_SIZE)

In [ ]:
file_name = '/kaggle/input/timm-pretrained-mobilenetv3/mobilenetv3/mobilenetv3_large_100_ra-f55367f5.pth'

model = timm.create_model('mobilenetv3_large_100', pretrained=False)

model.load_state_dict(torch.load(file_name))

model.reset_classifier(0)


In [ ]:
IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)



transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                transforms.Normalize(
                                    mean=torch.tensor(IMAGENET_DEFAULT_MEAN),
                                    std=torch.tensor(IMAGENET_DEFAULT_STD)),
                                transforms.RandomErasing()
                               ])

def load_image(file_name):
    file_path = f'/kaggle/input/shopee-product-matching/train_images/{file_name}'

    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    tensor_img = torch.tensor(img)
    tensor_img = tensor_img.permute(( 2, 0, 1)).float()
    tensor_img = transform(tensor_img)
    return tensor_img
    
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.anchor = df['anchor'].values
        self.positive = df['positive'].values
        self.negative = df['negative'].values
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        anchor = self.anchor[idx]
        positive = self.positive[idx]
        negative = self.negative[idx]
        
        anchor = load_image(anchor)
        positive = load_image(positive)
        negative = load_image(negative)


        
        return anchor, positive, negative

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.anchor = df['anchor'].values

        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        anchor = self.anchor[idx]
        anchor = load_image(anchor)
        return anchor

In [ ]:
train_dataset = TrainDataset(anchor_pos_neg)
train_loader = DataLoader(train_dataset,
                         batch_size=200,
                         shuffle=True,
                         num_workers=4,
                         pin_memory=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.TripletMarginWithDistanceLoss(distance_function=nn.CosineSimilarity())



In [ ]:
epochs =10
model.train()

scaler = torch.cuda.amp.GradScaler()

for epoch in tqdm(range(epochs), desc="Epochs"):
    running_loss = []
    for step, (anchor_img, positive_img, negative_img) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        anchor_img = anchor_img.to(device)
        positive_img = positive_img.to(device)
        negative_img = negative_img.to(device)
        
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            anchor_out = model(anchor_img)
            positive_out = model(positive_img)
            negative_out = model(negative_img)
        
            loss = criterion(anchor_out, positive_out, negative_out)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        
        running_loss.append(loss.cpu().detach().numpy())
    
    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, epochs, np.mean(running_loss)))
torch.save(model, './pretrained-model.pt')